In [21]:
data_path = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

In [22]:
from Tool_Functions.cleaning_data import * 

from Clustering.new_data_set_all import * 
from Clustering.viualize_datas import * 
from Clustering.new_data_set_all import *

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import matplotlib.pyplot as plt


In [24]:
def clean_new_df(data_path):
    df = file_to_dataframe(data_path + 'New_abos.csv')
    df = df.rename( columns= {'DAT_MVT_REELLE' : 'DATE_ACTE_REEL'})
    df['DATE_ACTE_REEL'] = pd.to_datetime(df['DATE_ACTE_REEL'])
    save_to_csv_file(df, data_path + 'new_abos.csv')
    return True
    

In [25]:
clean_new_df(data_path)

True

In [26]:
def upload_new_df(data_path):
    df_original = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_v2.csv")
    dfn = file_to_dataframe(data_path + 'New_abos.csv')
    df_test = join_dataFrames_outer(df_original,dfn, 'ID_ABONNE')
    df_test['DATE_ACTE_REEL_x'].fillna(df_test['DATE_ACTE_REEL_y'], inplace=True)
    df_test.drop(columns=['DATE_ACTE_REEL_y'], inplace=True)
    df_test = df_test.rename( columns= {'DATE_ACTE_REEL_x' : 'DATE_ACTE_REEL'})
    save_to_csv_file(df_test, data_path + 'total_df.csv')
    return True

In [27]:
upload_new_df(data_path)

/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Statapp-CANAL-1/Data_operations/Tool_Functions/join_data.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  datas = pd.read_csv(filename,delimiter=st)


True

In [31]:
def upload_new_df_clusters(data_path):
    df_test = file_to_dataframe(data_path + 'total_df.csv')
    df_id = file_to_dataframe(data_path + 'df_id.csv')
    df_merge = pd.merge(df_test, df_id, on='ID_ABONNE', how='left')
    df_merge['Cluster_8'].fillna('unclustered', inplace=True)
    save_to_csv_file(df_merge, data_path + 'total_df_clusters.csv')
    return True

In [32]:
upload_new_df_clusters(data_path)

/var/folders/md/_fklkl9s1z9gj6bpnwj204y40000gn/T/ipykernel_4990/1503170786.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'unclustered' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merge['Cluster_8'].fillna('unclustered', inplace=True)


True

In [33]:
df_truncated = file_to_dataframe(data_path + 'df_Données_Reabos_odd_final.csv')
df_original = file_to_dataframe(data_path + "df_Donnees_Reabos_odd_v2.csv")
dfn = file_to_dataframe(data_path + 'New_abos.csv')
df_test = file_to_dataframe(data_path + 'total_df.csv')
df_clusters = file_to_dataframe(data_path + 'df_all_clusters_final.csv')
df_id = file_to_dataframe(data_path + 'df_id.csv')
df_all_clusters = file_to_dataframe(data_path + 'total_df_clusters.csv')

/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Statapp-CANAL-1/Data_operations/Tool_Functions/join_data.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  datas = pd.read_csv(filename,delimiter=st)
